In [ ]:
!pip  install pandas
!pip  install numpy
!pip  install datetime
!pip  install regex
!pip  install openai
!pip  install bs4
!pip  install requests
!pip  install json
!pip  install numpy



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 14.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
import pandas as pd

# ==============================
# 1. Cargar archivo CSV
# ==============================
df = pd.read_csv("bq-results-20251023-000630-1761178021663.csv", encoding="utf-8-sig")

# Limpieza de encabezados para evitar errores por espacios o caracteres invisibles
df.columns = (
    df.columns.str.encode('utf-8').str.decode('utf-8', 'ignore')
    .str.replace("ï»¿", "", regex=False)
    .str.strip()
)

# ==============================
# 2. Limpieza y formato de fecha
# ==============================
# Verificar si la columna SQLDATE existe
col_fecha = [c for c in df.columns if "SQLDATE" in c.upper()]
if not col_fecha:
    raise ValueError("No se encontró la columna SQLDATE en el archivo CSV.")
col_fecha = col_fecha[0]

# Convertir fechas con detección automática
df[col_fecha] = pd.to_datetime(df[col_fecha].astype(str), errors='coerce')
df_2013 = df[df[col_fecha].dt.year == 2013].copy()

# ==============================
# 4. Filtro por país Colombia
# ==============================
col_pais = [c for c in df.columns if "COUNTRYCODE" in c.upper()]
if not col_pais:
    raise ValueError("No se encontró la columna ActionGeo_CountryCode en el archivo CSV.")
col_pais = col_pais[0]

df_colombia = df_2013[df_2013[col_pais].astype(str).str.upper() == "CO"].copy()

# ==============================
# 5. Filtro por contenido del SOURCEURL
# ==============================
col_url = [c for c in df.columns if "SOURCEURL" in c.upper()]
if not col_url:
    raise ValueError("No se encontró la columna SOURCEURL en el archivo CSV.")
col_url = col_url[0]

df_colombia[col_url] = df_colombia[col_url].astype(str).str.lower()

palabras_clave = ['paro-docente', 'fecode', 'maestros', 'protesta-docente', 'paro-maestros']

filtro_tema = df_colombia[col_url].apply(
    lambda x: any(palabra in x for palabra in palabras_clave)
)

df_filtrado = df_colombia.loc[filtro_tema, [col_fecha, 'ActionGeo_FullName', col_url]].copy()

# ==============================
# 6. Eliminar duplicados
# ==============================
df_filtrado = df_filtrado.drop_duplicates(subset=[col_url], keep='first')

# ==============================
# 7. Guardar resultado final
# ==============================
print("Noticias filtradas:", len(df_filtrado))
print(df_filtrado.head())

df_filtrado.to_csv("BQ_Noticias_Paro_Docente_2013.csv", index=False, encoding="utf-8-sig")


Noticias filtradas: 0
Empty DataFrame
Columns: [SQLDATE, ActionGeo_FullName, SOURCEURL]
Index: []


In [ ]:
import pandas as pd

# === 1. Cargar el archivo ===
df = pd.read_csv("Base-SGI-suspensión-clases-2012-2025-03_08_2025_VALIDACION-SED.csv")

# Normalizar nombres de columnas (elimina espacios, mayúsculas y tildes)
df.columns = df.columns.str.strip().str.lower().str.replace("á", "a").str.replace("é", "e").str.replace("í", "i").str.replace("ó", "o").str.replace("ú", "u")

# === 2. Unificar formato de fechas ===
for col in df.columns:
    if "fecha" in col:
        df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)

# === 3. Filtrar por año  2015  ===
df_2015 = df[
    (df["fecha inicio"].dt.year == 2013) |
    (df["fecha fin"].dt.year == 2013)
]


# Seleccionar las más relevantes si existen
cols_posibles = [
    "fecha inicio", "fecha fin", "convocante", "hecho","dias suspension clases",
    "ubicacion", "medio de verificacion","medio de verificacion 2", "tipo de protesta", "alcance"
]
cols_finales = [c for c in cols_posibles if c in df_2015.columns]

df_resultado = df_2015[cols_finales].reset_index(drop=True)
if "medio de verificacion" in df_resultado.columns:
    sin_info = df_resultado[df_resultado["medio de verificacion"].str.strip().str.upper() == "S/I"]
    con_info = df_resultado[df_resultado["medio de verificacion"].str.strip().str.upper() != "S/I"]
    con_info = con_info.drop_duplicates(subset=["medio de verificacion"], keep='first')
    df_resultado = pd.concat([con_info, sin_info], ignore_index=True)

# === 8. (Opcional) Guardar en CSV ===
df_resultado.to_csv("SED_Paros_Docentes_Colombia_2013.csv", index=False)


In [ ]:
import pandas as pd

# === 1. Cargar archivo ===
df = pd.read_csv("web_noticias_paros_docentes_2013.csv")

# === 2. Estandarizar nombres de columnas ===
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace("á", "a")
    .str.replace("é", "e")
    .str.replace("í", "i")
    .str.replace("ó", "o")
    .str.replace("ú", "u")
)

# === 3. Convertir la columna de fecha_publicacion a formato fecha y usarla como fecha_paro ===
df["fecha_publicacion"] = pd.to_datetime(df["fecha_publicacion"], errors="coerce", dayfirst=True)
df["fecha_paro"] = df["fecha_publicacion"]  # Usar fecha_publicacion como fecha_paro

# === 4. Filtrar solo los paros del año 2013 ===
df_2013 = df[df["fecha_paro"].dt.year == 2013]

# === 5. Seleccionar columnas relevantes ===
cols_finales = [
    "titulo",
    "fuente",
    "url",
    "fecha_paro",
    "duracion_dias",
    "organizaciones_sindicales",
    "razones_paro",
    "resumen",
    "ubicacion",
    "tipo_movilizacion"
]

df_reducido = df_2013[cols_finales].reset_index(drop=True)

# === 6. Exportar a CSV ===
df_reducido.to_csv("WEB_Paros_Docentes_2013_Reducido.csv", index=False)


/tmp/ipython-input-2875457156.py:19: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["fecha_publicacion"] = pd.to_datetime(df["fecha_publicacion"], errors="coerce", dayfirst=True)


In [ ]:
!pip install openai
!pip install os
!pip install pandas
!pip install numpy
!pip install requests
!pip install json
!pip install bs4
!pip install datetime
!pip install time
!pip install urllib.parse
!pip install difflib

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
ERROR: Could not find a version that satisfies the requirement urllib.parse (from versions: none)
ERROR: No matching distribution found for urllib.parse
ERROR: Could not find a version that satisfies the requirement difflib (from versions: none)
ERROR: No matching distribution found for difflib


In [ ]:
import os, re
import pandas as pd, numpy as np, requests, json, time
from bs4 import BeautifulSoup
from datetime import datetime
from openai import OpenAI
from urllib.parse import urlparse, parse_qsl, urlencode, urlunparse

os.environ["OPENAI_API_KEY"] = "

In [ ]:
FILE_WEB = "WEB_Paros_Docentes_2013_Reducido.csv"
FILE_SED = "SED_Paros_Docentes_Colombia_2013.csv"
FILE_BQ  = "BQ_Noticias_Paro_Docente_2013.csv"

# -------------------- UTILIDADES --------------------
def norm_cols(df):
    df.columns = (df.columns.str.strip().str.lower()
                  .str.replace("á","a").str.replace("é","e")
                  .str.replace("í","i").str.replace("ó","o").str.replace("ú","u"))
    return df

def safe_dt(x, dayfirst=False):
    return pd.to_datetime(x, errors="coerce", dayfirst=dayfirst)

def get_col(df, keys):
    for k in keys:
        hits = [c for c in df.columns if k in c]
        if hits: return hits[0]
    return None

def ensure_cols(df, cols, fill=np.nan):
    for c in cols:
        if c not in df.columns:
            df[c] = fill
    return df

# -------------------- CARGA --------------------
web = norm_cols(pd.read_csv(FILE_WEB))
sed = norm_cols(pd.read_csv(FILE_SED))
# Check if BQ file is empty before loading
try:
    bq = norm_cols(pd.read_csv(FILE_BQ))
except pd.errors.EmptyDataError:
    print(f"Warning: File {FILE_BQ} is empty. Skipping BQ data.")
    bq = pd.DataFrame()


for df in [web, sed, bq]:
    if not df.empty:
        for c in df.columns:
            if "fecha" in c:
                df[c] = safe_dt(df[c], dayfirst=False)

# ================================================================
# BASE SED
# ================================================================
col_fecha = get_col(sed, ["fecha inicio","fechainicio","fecha"])
col_fin   = get_col(sed, ["fecha fin","fechafin","fin"])
col_hecho = get_col(sed, ["hecho","evento","descripcion"])
col_tipo  = get_col(sed, ["tipo de protesta","tipo","movilizacion"])
col_conv  = get_col(sed, ["convocante","organizacion","sindicato"])
col_ubic  = get_col(sed, ["ubicacion","lugar","region"])
col_dias  = get_col(sed, ["dias suspension","duracion_dias","dias","duracion"])

if not col_dias and col_fecha and col_fin:
    sed["duracion_dias"] = (safe_dt(sed[col_fin]) - safe_dt(sed[col_fecha])).dt.days + 1
    sed.loc[sed["duracion_dias"] < 1, "duracion_dias"] = 1
    col_dias = "duracion_dias"
elif not col_dias:
    sed["duracion_dias"] = np.nan
    col_dias = "duracion_dias"

sel = [c for c in [col_fecha,col_hecho,col_tipo,col_conv,col_ubic,col_dias] if c]
sed_final = sed[sel].rename(columns={
    col_fecha:"fecha_evento",
    col_hecho:"razones_paro",
    col_tipo:"tipo_movilizacion",
    col_conv:"organizaciones_sindicales",
    col_ubic:"region",
    col_dias:"duracion_dias"
}).copy()

std = ["fecha_evento","region","organizaciones_sindicales","tipo_movilizacion",
       "duracion_dias","razones_paro","resumen","titulo_ref","urls","fuentes_presentes"]
sed_final = ensure_cols(sed_final, std)
sed_final["resumen"] = sed_final["razones_paro"]
sed_final["titulo_ref"] = sed_final["razones_paro"].astype(str).str[:90]
sed_final["fuentes_presentes"] = "SED"

col_url_sed = get_col(sed, ["medio de verificacion","url","enlace"])
sed_final["urls"] = sed[col_url_sed] if col_url_sed else np.nan

# ================================================================
#  BASE WEB
# ================================================================
web_final = web.rename(columns={"fecha_paro":"fecha_evento","ubicacion":"region"}).copy()
web_final = ensure_cols(web_final, std)
web_final["fuentes_presentes"] = "WEB"
if "url" in web_final.columns: web_final["urls"] = web_final["url"]
if "titulo" in web_final.columns: web_final["titulo_ref"] = web_final["titulo"]

# ================================================================
#  BASE BQ
# ================================================================
bq_final = pd.DataFrame()
if not bq.empty:
    # Normalizar encabezados
    bq.columns = (
        bq.columns.str.encode('utf-8').str.decode('utf-8', 'ignore')
        .str.replace("ï»¿", "", regex=False)
        .str.strip()
        .str.lower()
    )

    col_sqldate = [c for c in bq.columns if "sqldate" in c]
    col_url_bq = [c for c in bq.columns if "sourceurl" in c or "url" in c]
    col_region_bq = [c for c in bq.columns if "actiongeo_fullname" in c or "region" in c]

    if not col_sqldate:
        print("Warning: No se encontró la columna SQLDATE en la base BQ.")
        bq_final = pd.DataFrame(columns=std) # Create empty df with standard columns
    else:
        col_sqldate = col_sqldate[0]
        col_url_bq = col_url_bq[0] if col_url_bq else None
        col_region_bq = col_region_bq[0] if col_region_bq else None

        def parse_fecha(f):
            if pd.isna(f):
                return pd.NaT
            f = str(f).strip()
            for fmt in ("%Y%m%d", "%d/%m/%Y", "%Y-%m-%d"):
                try:
                    return pd.to_datetime(f, format=fmt)
                except Exception:
                    continue
            return pd.to_datetime(f, errors="coerce")

        bq["fecha_evento"] = bq[col_sqldate].apply(parse_fecha)

        # Crear DataFrame final con columnas estandarizadas
        bq_final = pd.DataFrame({
            "fecha_evento": bq["fecha_evento"],
            "urls": bq[col_url_bq] if col_url_bq else np.nan,
            "region": bq[col_region_bq] if col_region_bq else np.nan,
            "organizaciones_sindicales": bq.get("organizaciones_sindicales", np.nan),
            "tipo_movilizacion": bq.get("tipo_movilizacion", "paro docente"),
            "razones_paro": bq.get("razones_paro", np.nan),
            "resumen": bq.get("resumen", np.nan),
            "duracion_dias": bq.get("duracion_dias", np.nan),
            "titulo_ref": bq.get("titulo", np.nan)
        })

        bq_final = ensure_cols(bq_final, std)
        bq_final["fuentes_presentes"] = "BQ"


# ================================================================
# ESTANDARIZAR LOCALIZACIÓN
# ================================================================
CIUDADES_CO = {
    "bogota":"Bogotá","bogotá":"Bogotá","medellin":"Medellín","medellín":"Medellín",
    "cali":"Cali","barranquilla":"Barranquilla","cartagena":"Cartagena","pereira":"Pereira",
    "armenia":"Armenia","manizales":"Manizales","bucaramanga":"Bucaramanga","cucuta":"Cúcuta",
    "cúcuta":"Cúcuta","ibague":"Ibagué","ibagué":"Ibagué","villavicencio":"Villavicencio",
    "santa marta":"Santa Marta","monteria":"Montería","neiva":"Neiva","tunja":"Tunja",
    "yopal":"Yopal","pasto":"Pasto","popayan":"Popayán","riohacha":"Riohacha","quibdo":"Quibdó",
    "leticia":"Leticia","mocoa":"Mocoa"
}

def _clean(s):
    if s is None or (isinstance(s,float) and pd.isna(s)): return ""
    return str(s).strip()

def parse_region_to_pais_ciudad(val):
    v = _clean(val)
    if v == "" or v.upper() in {"S/I","S. I.","NA","N/A"}:
        return ("Colombia","")
    if "colombia" in v.lower():
        partes = [p.strip() for p in re.split(r",", v) if p.strip()]
        if len(partes) >= 2:
            return ("Colombia", CIUDADES_CO.get(partes[0].lower(), partes[0]))
    for k, pretty in CIUDADES_CO.items():
        if re.search(rf"\b{k}\b", v.lower()):
            return ("Colombia", pretty)
    return ("Colombia", v[:80])

def add_pais_ciudad(df):
    if df.empty:
        df["pais"] = []
        df["ciudad"] = []
        return df
    p, c = zip(*df["region"].apply(parse_region_to_pais_ciudad))
    df["pais"], df["ciudad"] = p, c
    return df

sed_final = add_pais_ciudad(sed_final)
web_final = add_pais_ciudad(web_final)
bq_final  = add_pais_ciudad(bq_final)

# ================================================================
#  UNIR BASES
# ================================================================
cols_union = ["fecha_evento","pais","ciudad","organizaciones_sindicales","tipo_movilizacion",
              "duracion_dias","razones_paro","resumen","titulo_ref","urls","fuentes_presentes","region"]

for df in [sed_final, web_final, bq_final]:
    if not df.empty:
        for c in cols_union:
            if c not in df.columns: df[c] = np.nan
    else: # Ensure empty dfs have the required columns before concat
         for c in cols_union:
            if c not in df.columns:
                df[c] = pd.Series([], dtype='object') # Use object dtype for mixed types


base = pd.concat([sed_final[cols_union], web_final[cols_union], bq_final[cols_union]], ignore_index=True)
base = base.sort_values(["fecha_evento","ciudad"]).reset_index(drop=True)

# ================================================================
# UNIFICAR Y LIMPIAR ORGANIZACIONES SINDICALES
# ================================================================
def estandarizar_orgs(valor):
    """Limpia, normaliza y unifica nombres de sindicatos."""
    if pd.isna(valor):
        return "NO ESPECÍFICA"

    if isinstance(valor, (list, tuple, set)):
        texto = " | ".join(map(str, valor))
    else:
        texto = str(valor)

    texto = re.sub(r"[\[\]'\"{}]", "", texto).upper()
    partes = re.split(r"[;|,/\-]+", texto)
    partes = [p.strip() for p in partes if p.strip()]
    if not partes:
        return "NO ESPECÍFICA"

    equivalencias = {
        "FEDERACION COLOMBIANA DE TRABAJADORES DE LA EDUCACION": "FECODE",
        "FECODE": "FECODE",
        "ADE": "ADE",
        "ADIDA": "ADIDA",
        "CUT": "CUT",
        "CENTRAL UNITARIA DE TRABAJADORES": "CUT",
        "CENTRAL UNITARIA DE TRABAJADORES DE COLOMBIA": "CUT",
        "CONFEDERACION DE TRABAJADORES DE COLOMBIA": "CTC",
        "CTC": "CTC",
        "CGT": "CGT",
        "CONFEDERACION GENERAL DEL TRABAJO": "CGT",
        "SINTRANAL": "SINTRANAL",
        "SINTRANAL EDU": "SINTRANAL",
        "NO ESPECIFICA": "NO ESPECÍFICA",
        "NO ESPECÍFICA": "NO ESPECÍFICA"
    }

    normalizados = []
    for p in partes:
        p_clean = re.sub(r"[^A-ZÁÉÍÓÚÜÑ ]", "", p).strip()
        for k, v in equivalencias.items():
            if k in p_clean:
                normalizados.append(v)
                break
        else:
            normalizados.append(p_clean)

    limpio = [x for x in sorted(set(normalizados)) if x]
    if not limpio:
        return "NO ESPECÍFICA"
    return " | ".join(limpio)

if not base.empty:
    base["organizaciones_sindicales"] = base["organizaciones_sindicales"].apply(estandarizar_orgs)

# ================================================================
# ESTADÍSTICAS DESCRIPTIVAS
# ================================================================
stats = {}
if not base.empty:
    stats = {
        "Total_paros": len(base),
        "Promedio_duracion": round(base["duracion_dias"].mean(skipna=True), 2),
        "Max_duracion": base["duracion_dias"].max(),
        "Min_duracion": base["duracion_dias"].min(),
        "Tipos_movilizacion": base["tipo_movilizacion"].value_counts().to_dict(),
        "Ciudades_principales": base["ciudad"].value_counts().head(5).to_dict(),
        "Sindicatos_principales": base["organizaciones_sindicales"].value_counts().head(5).to_dict(),
        "Fuentes": base["fuentes_presentes"].value_counts().to_dict()
    }
    pd.DataFrame([stats]).to_csv("ESTADISTICAS_PAROS_DOCENTES_2015.csv", index=False, encoding="utf-8-sig")
else:
    print("\nNo data to generate statistics.")


# ================================================================
# EXPORTAR BASE FINAL
# ================================================================
cols_export = ["fecha_evento","pais","ciudad","organizaciones_sindicales","tipo_movilizacion",
               "duracion_dias","razones_paro","resumen","titulo_ref","urls","fuentes_presentes"]

if not base.empty:
    base["fecha_evento"] = pd.to_datetime(base["fecha_evento"], errors="coerce").dt.strftime("%Y-%m-%d")
    base[cols_export].to_csv("BASE_FINAL_PAROS_DOCENTES_2023_VALIDADA.csv", index=False, encoding="utf-8-sig")
    print("\n✅ BASE_FINAL_PAROS_DOCENTES_2023_VALIDADA.csv generada correctamente")
    print("\nResumen general:")
    for k, v in stats.items():
        print(f"- {k}: {v}")
else:
    print("\nNo data to export BASE_FINAL_PAROS_DOCENTES_2023_VALIDADA.csv")


✅ BASE_FINAL_PAROS_DOCENTES_2023_VALIDADA.csv generada correctamente

Resumen general:
- Total_paros: 11
- Promedio_duracion: 6.0
- Max_duracion: 18.0
- Min_duracion: 1.0
- Tipos_movilizacion: {'paro docente': 6, 'Paro': 3, 'Sin evidencia': 1, 'Movilización contenida en paro': 1}
- Ciudades_principales: {'Colombia': 6, '': 3, 'Secretaría de Educación (Av. El Dorado) y finalizó frente al Ministerio de Educa': 1, 'Bogotá': 1}
- Sindicatos_principales: {'ADE | FECODE': 5, 'NO ESPECÍFICA': 2, 'CUT | FECODE': 2, 'FECODE': 1, 'SUTEV': 1}
- Fuentes: {'WEB': 6, 'SED': 5}


/tmp/ipython-input-605807243.py:206: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  base = pd.concat([sed_final[cols_union], web_final[cols_union], bq_final[cols_union]], ignore_index=True)


In [ ]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
import json

# ================================================================
# CARGA DE BASE
# ================================================================
df = pd.read_csv("BASE_FINAL_PAROS_DOCENTES_2023_VALIDADA.csv", encoding="utf-8-sig")
df.columns = df.columns.str.strip().str.lower()

# ================================================================
# FUNCIONES AUXILIARES
# ================================================================
def limpiar_texto(texto):
    if pd.isna(texto):
        return ""
    return re.sub(r"\s+", " ", str(texto)).strip()

def obtener_contenido_web(url):
    if not isinstance(url, str) or url.strip().upper() in ["", "S/I", "N/A"]:
        return ""
    try:
        r = requests.get(url, timeout=15)
        if r.status_code != 200:
            return ""
        soup = BeautifulSoup(r.text, "html.parser")
        for tag in soup(["script", "style", "noscript", "footer", "header", "aside"]):
            tag.extract()
        texto = " ".join(soup.stripped_strings)
        return texto[:8000]
    except Exception:
        return ""

# ================================================================
# ANALIZAR NOTICIA CON IA (GPT-4o MINI)
# ================================================================
def analizar_noticia_ia(fila):
    url = fila.get("urls", "")
    titulo = fila.get("titulo_ref", "")
    texto = obtener_contenido_web(url)

    prompt = f"""
Estandariza y completa la información de una noticia sobre paros docentes en Colombia.

Entrada:
Título: {titulo}
Texto de la noticia: {texto[:1500]}

Devuelve un JSON válido con las siguientes claves:
{{
  "ubicacion_bogota": true/false → si ocurre en Bogotá/Cundinamarca,
  "ubicacion_colombia": true/false → si el paro abarca el territorio Nacional,
  "ciudad": "nombre de la ciudad (si se menciona o vacía si no)",
  "tipo_movilizacion": "paro docente / plantón / marcha / toma / protesta / asamblea",
  "duracion_dias": número o null,
  "razones_paro": "resumen breve de motivos (salud, reformas, infraestructura, etc.)",
  "resumen": "síntesis breve del hecho (máximo 3 líneas)",
  "fecha_hecho": "fecha más probable del hecho en formato YYYY-MM-DD o vacío si no se menciona"
}}

Si la información no está explícita, infiere con contexto general sobre paros docentes en Colombia.
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0.2,
            messages=[{"role": "user", "content": prompt}],
        )
        salida = response.choices[0].message.content.strip()
        try:
            datos = json.loads(salida)
        except:
            match = re.search(r"\{.*\}", salida, re.DOTALL)
            datos = json.loads(match.group(0)) if match else {}
        return datos

    except Exception as e:
        print(f"Error IA: {e}")
        return {
            "ubicacion_bogota": None,
            "ubicacion_colombia": None,
            "ciudad": "",
            "tipo_movilizacion": "",
            "duracion_dias": None,
            "razones_paro": "",
            "resumen": "",
            "fecha_hecho": ""
        }

# ================================================================
# DIVIDIR ENTRE ANALIZABLES Y NO ANALIZABLES
# ================================================================
df["fuentes_presentes"] = df["fuentes_presentes"].astype(str).str.upper().str.strip()
mask_ia = df["fuentes_presentes"].isin(["SED", "BQ"]) & df["urls"].notna() & ~df["urls"].str.upper().isin(["S/I", "N/A", ""])
df_analizar = df[mask_ia].copy()
df_conservar = df[~mask_ia].copy()

print(f"Noticias a analizar con IA: {len(df_analizar)}")
print(f"Noticias que se conservarán sin análisis IA: {len(df_conservar)}")

# ================================================================
# PROCESAR CON IA Y RELLENAR SI FALTA INFO
# ================================================================
resultados_ia = []

for i, fila in enumerate(df_analizar.to_dict("records")):
    print(f"[{i+1}/{len(df_analizar)}] Analizando IA: {fila.get('urls', '')[:80]}")
    try:
        analisis = analizar_noticia_ia(fila)

        # Si la IA no devuelve fecha_hecho, usar 'fecha_evento'
        if not analisis.get("fecha_hecho"):
            analisis["fecha_hecho"] = fila.get("fecha_evento", "")

        # Si la IA no devuelve duracion_dias, usar 'duracion_dias' de la base
        if not analisis.get("duracion_dias"):
            try:
                dias = int(fila.get("duracion_dias", 0))
                analisis["duracion_dias"] = dias if dias > 0 else None
            except:
                analisis["duracion_dias"] = None

        fila.update(analisis)
        resultados_ia.append(fila)

    except Exception as e:
        print(f"Error en fila {i+1}: {e}")
        resultados_ia.append(fila)

    time.sleep(2)

df_ia = pd.DataFrame(resultados_ia)

# ================================================================
# UNIR TODO Y LIMPIAR COLUMNAS AUXILIARES
# ================================================================
df_final = pd.concat([df_conservar, df_ia], ignore_index=True)

columnas_a_eliminar = ["ubicacion_bogota", "ubicacion_colombia", "ciudad", "fecha_hecho"]
df_final = df_final.drop(columns=[col for col in columnas_a_eliminar if col in df_final.columns])

# === Guardar resultado si se desea ===
# df_final.to_csv("Paros_Docentes_Analizados.csv", index=False, encoding="utf-8-sig")


Noticias a analizar con IA: 5
Noticias que se conservarán sin análisis IA: 6
[1/5] Analizando IA: https://groups.google.com/g/mesa-distrital-orientadores-escolares-/c/SPAiGj6kOUc
[2/5] Analizando IA: https://www.pares.com.co/post/impactos-que-tuvo-el-paro-agrario?utm_source
[3/5] Analizando IA: https://ocecolombia.co/movilizacion-distrital-de-la-ade-el-7-de-febrero/
[4/5] Analizando IA: https://www.bbc.com/mundo/ultimas_noticias/2013/09/130910_ultnot_educacion_colom
[5/5] Analizando IA: https://elpais.com/economia/2013/09/11/agencias/1378931240_545234.html?utm_sourc


In [ ]:
# ================================================================
# ESTANDARIZAR ORGANIZACIONES SINDICALES
# ================================================================
def estandarizar_orgs(valor):
    """Estandariza nombres de sindicatos en formato único y consistente."""
    # Manejar valores vacíos, listas o arrays
    if valor is None:
        return ""
    if isinstance(valor, (list, set, tuple)):
        texto = " ".join(map(str, valor))
    elif isinstance(valor, (pd.Series, np.ndarray)):
        texto = " ".join(map(str, valor.tolist()))
    else:
        texto = str(valor)

    texto = texto.strip()
    if texto == "" or texto.upper() in ["NAN", "NONE"]:
        return "NO ESPECÍFICA"

    texto = re.sub(r"[\[\]'\"{}]", "", texto)
    partes = re.split(r"[;|,/\-]+", texto)
    partes = [p.strip().upper() for p in partes if p.strip()]

    equivalencias = {
        "ADE": ["ADE", "ASOCIACION DISTRITAL DE EDUCADORES", "ASOCIACIÓN DISTRITAL DE EDUCADORES"],
        "FECODE": ["FECODE", "FEDERACION COLOMBIANA DE TRABAJADORES DE LA EDUCACION", "FEDERACIÓN COLOMBIANA DE TRABAJADORES DE LA EDUCACIÓN"],
        "CUT": ["CUT", "CENTRAL UNITARIA DE TRABAJADORES"],
        "CTC": ["CTC", "CONFEDERACION DE TRABAJADORES DE COLOMBIA", "CONFEDERACIÓN DE TRABAJADORES DE COLOMBIA"],
        "SINTRANAL": ["SINTRANAL", "SINDICATO NACIONAL DE TRABAJADORES DE LA EDUCACION", "SINDICATO NACIONAL DE TRABAJADORES DE LA EDUCACIÓN"],
        "ADIDA": ["ADIDA", "ASOCIACION DE INSTITUTO DE ANTIOQUIA", "ASOCIACIÓN DE INSTITUTO DE ANTIOQUIA"]
    }

    normalizados = set()
    for p in partes:
        for clave, variantes in equivalencias.items():
            if any(v in p for v in variantes):
                normalizados.add(clave)
                break
        else:
            if p and p not in ["NO ESPECIFICA", "NO ESPECÍFICA", "NO APLICA"]:
                normalizados.add(p)

    return " | ".join(sorted(normalizados)) if normalizados else "NO ESPECÍFICA"


df_final ["organizaciones_sindicales"] = df_final["organizaciones_sindicales"].apply(estandarizar_orgs)

# ================================================================
# GUARDAR RESULTADO FINAL
# ================================================================
df_final.to_csv("BASE_PAROS_DOCENTES_2023_IA.csv", index=False, encoding="utf-8-sig")

print("Proceso completado correctamente.")
print("Archivo generado: BASE_PAROS_DOCENTES_2023_IA.csv")


Proceso completado correctamente.
Archivo generado: BASE_PAROS_DOCENTES_2023_IA.csv


In [ ]:
# ================================================================
# UNIFICAR PAROS DOCENTES 2015  (duración desde inicio/fin si falta)
# ================================================================
import pandas as pd
import numpy as np
import re
from datetime import timedelta

# -----------------------------
# 1. CARGAR BASE
# -----------------------------
# Si tu archivo fuente ya es de 2015, usa ese nombre:
# df = pd.read_csv("BASE_PAROS_DOCENTES_2015_IA.csv", encoding="utf-8-sig")
# Si aún usas el de 2023 pero filtrado a 2015, déjalo así:
df = pd.read_csv("BASE_PAROS_DOCENTES_2023_IA.csv", encoding="utf-8-sig")
df.columns = df.columns.str.strip().str.lower()

if "fecha_evento" in df.columns:
    df["fecha_evento"] = pd.to_datetime(df["fecha_evento"], errors="coerce")

df = df.dropna(subset=["fecha_evento"]).sort_values("fecha_evento").reset_index(drop=True)

# -----------------------------
# 2. FUNCIONES AUXILIARES
# -----------------------------
def combinar_textos(series):
    """Combina textos o listas de textos sin duplicar información."""
    textos = []
    for val in series.dropna():
        if isinstance(val, str):
            partes = re.split(r"[|;]+", val)
            textos.extend([p.strip() for p in partes if p.strip()])
        elif isinstance(val, (list, set, tuple)):
            textos.extend([str(p).strip() for p in val if str(p).strip()])
    return "; ".join(sorted(set(textos)))

def combinar_numeros(series):
    """Promedio de valores numéricos."""
    vals = pd.to_numeric(series, errors="coerce")
    return round(vals.mean(skipna=True), 1) if vals.notna().any() else np.nan

def combinar_fuentes(series):
    """Une todas las fuentes sin repetir."""
    return "; ".join(sorted(set(
        re.split(r"[;|,]+", ";".join(series.dropna().astype(str)))
    )))

def combinar_urls(series):
    """Combina todas las URLs únicas, creando columnas url_1, url_2, ..."""
    urls = []
    for val in series.dropna():
        if isinstance(val, str):
            for u in re.split(r"[|,;]+", val):
                if u.strip() and u.strip().startswith("http"):
                    urls.append(u.strip())
    return sorted(set(urls))

# -----------------------------
# 3. AGRUPAR POR FECHA CON TOLERANCIA DE ±1 DÍA (mismo evento)
# -----------------------------
df = df.sort_values("fecha_evento").reset_index(drop=True)

grupos = []
grupo_actual = 0
fechas = df["fecha_evento"].tolist()

for i in range(len(fechas)):
    if i == 0:
        grupos.append(grupo_actual)
        continue
    diff = (fechas[i] - fechas[i - 1]).days
    if diff <= 3:
        grupos.append(grupo_actual)
    else:
        grupo_actual += 1
        grupos.append(grupo_actual)

df["grupo_paro"] = grupos

# -----------------------------
# 4. FUSIONAR INFORMACIÓN POR GRUPO
# -----------------------------
agrupado = []
for gid, sub in df.groupby("grupo_paro"):
    fila = {}
    fila["fecha_inicio"] = sub["fecha_evento"].min()
    fila["fecha_fin"]   = sub["fecha_evento"].max()
    fila["ciudad"] = combinar_textos(sub.get("ciudad", pd.Series([], dtype=object)))
    fila["organizaciones_sindicales"] = combinar_textos(sub.get("organizaciones_sindicales", pd.Series([], dtype=object)))
    fila["tipo_movilizacion"] = combinar_textos(sub.get("tipo_movilizacion", pd.Series([], dtype=object)))
    # Promedio si venía reportado; si no, luego lo recalculamos desde fechas
    fila["duracion_dias"] = combinar_numeros(sub.get("duracion_dias", pd.Series([], dtype=object)))
    fila["razones_paro"] = combinar_textos(sub.get("razones_paro", pd.Series([], dtype=object)))
    fila["resumen"] = combinar_textos(sub.get("resumen", pd.Series([], dtype=object)))
    fila["fuentes_presentes"] = combinar_fuentes(sub.get("fuentes_presentes", pd.Series([], dtype=object)))

    urls_unicas = combinar_urls(sub.get("urls", pd.Series([], dtype=object)))
    for i, url in enumerate(urls_unicas, start=1):
        fila[f"url_{i}"] = url

    agrupado.append(fila)

base_unificada = pd.DataFrame(agrupado).sort_values("fecha_inicio").reset_index(drop=True)

# -----------------------------
# 4.1 CALCULAR DURACIÓN DESDE INICIO/FIN **solo si duracion_dias está vacía**
# -----------------------------
# Parse seguro de fechas (en caso de venir como string)
base_unificada["fecha_inicio"] = pd.to_datetime(base_unificada["fecha_inicio"], errors="coerce")
base_unificada["fecha_fin"]    = pd.to_datetime(base_unificada["fecha_fin"], errors="coerce")

# Máscara: filas donde duracion_dias está NaN y SÍ tenemos ambas fechas
mask_calc = base_unificada["duracion_dias"].isna() & base_unificada["fecha_inicio"].notna() & base_unificada["fecha_fin"].notna()

# Diferencia inclusiva: si inicio == fin -> 1 día; si no, (fin - inicio).days + 1
duracion_calc = (base_unificada.loc[mask_calc, "fecha_fin"] - base_unificada.loc[mask_calc, "fecha_inicio"]).dt.days + 1
# Evitar valores no positivos por si hay desorden en fechas (forzar mínimo 1)
duracion_calc = duracion_calc.clip(lower=1)

# Asignar solo a las filas que cumplen la máscara
base_unificada.loc[mask_calc, "duracion_dias"] = duracion_calc.astype("Int64")

# -----------------------------
# 5. EXPORTAR RESULTADO
# -----------------------------
# Salida en formato ISO
base_unificada["fecha_inicio"] = base_unificada["fecha_inicio"].dt.strftime("%Y-%m-%d")
base_unificada["fecha_fin"]    = base_unificada["fecha_fin"].dt.strftime("%Y-%m-%d")

base_unificada.to_csv("BASE_PAROS_UNIFICADOS_2015.csv", index=False, encoding="utf-8-sig")

print("Base unificada generada: BASE_PAROS_UNIFICADOS_2013.csv")
print("Paros únicos encontrados:", len(base_unificada))
print("Número máximo de columnas URL:", base_unificada.filter(like='url_').shape[1])
print("Filas con duración calculada por fechas:", mask_calc.sum())


Base unificada generada: BASE_PAROS_UNIFICADOS_2015.csv
Paros únicos encontrados: 9
Número máximo de columnas URL: 2
Filas con duración calculada por fechas: 2


In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta

# -----------------------------
# 1. CARGAR BASE
# -----------------------------
df = pd.read_csv("BASE_FINAL_PAROS_DOCENTES_2023_VALIDADA.csv", encoding="utf-8-sig")
df.columns = df.columns.str.strip().str.lower()

# Normalizar fechas
df["fecha_evento"] = pd.to_datetime(df["fecha_evento"], errors="coerce")
df = df.dropna(subset=["fecha_evento"]).sort_values("fecha_evento").reset_index(drop=True)

# -----------------------------
# 2. FUNCIONES AUXILIARES
# -----------------------------
def combinar_textos(series):
    textos = []
    for val in series.dropna():
        if isinstance(val, str):
            partes = re.split(r"[|;]+", val)
            textos.extend([p.strip() for p in partes if p.strip()])
        elif isinstance(val, (list, set, tuple)):
            textos.extend([str(p).strip() for p in val if str(p).strip()])
    return "; ".join(sorted(set(textos)))

def combinar_numeros(series):
    vals = pd.to_numeric(series, errors="coerce")
    return round(vals.mean(skipna=True), 1) if vals.notna().any() else np.nan

def combinar_fuentes(series):
    return "; ".join(sorted(set(re.split(r"[;|,]+", ";".join(series.dropna().astype(str))))))

def combinar_urls(series):
    urls = []
    for val in series.dropna():
        if isinstance(val, str):
            for u in re.split(r"[|,;]+", val):
                if u.strip() and u.strip().startswith("http"):
                    urls.append(u.strip())
    return sorted(set(urls))

# -----------------------------
# 3. AGRUPAR POR FECHA CON TOLERANCIA DE ±3 DÍAS
# -----------------------------
df = df.sort_values("fecha_evento").reset_index(drop=True)
grupos = []
grupo_actual = 0
fechas = df["fecha_evento"].tolist()

for i in range(len(fechas)):
    if i == 0:
        grupos.append(grupo_actual)
        continue
    diff = (fechas[i] - fechas[i - 1]).days
    if diff <= 3:
        grupos.append(grupo_actual)
    else:
        grupo_actual += 1
        grupos.append(grupo_actual)

df["grupo_paro"] = grupos

# -----------------------------
# 4. FUSIONAR INFORMACIÓN POR GRUPO Y CALCULAR FECHA FIN
# -----------------------------
agrupado = []
for gid, sub in df.groupby("grupo_paro"):
    fila = {}

    # Fecha de inicio: mínima
    fecha_inicio = sub["fecha_evento"].min()

    # Duración: promedio o 1 como mínimo
    duracion_dias = combinar_numeros(sub.get("duracion_dias", pd.Series([], dtype=object)))
    if pd.isna(duracion_dias) or duracion_dias < 1:
        duracion_dias = 1

    # Fecha de fin = fecha_inicio + duración - 1
    fecha_fin = fecha_inicio + timedelta(days=int(duracion_dias) - 1)

    fila["fecha_inicio"] = fecha_inicio
    fila["fecha_fin"] = fecha_fin
    fila["duracion_dias"] = int(duracion_dias)

    fila["ciudad"] = combinar_textos(sub.get("ciudad", pd.Series([], dtype=object)))
    fila["organizaciones_sindicales"] = combinar_textos(sub.get("organizaciones_sindicales", pd.Series([], dtype=object)))
    fila["tipo_movilizacion"] = combinar_textos(sub.get("tipo_movilizacion", pd.Series([], dtype=object)))
    fila["razones_paro"] = combinar_textos(sub.get("razones_paro", pd.Series([], dtype=object)))
    fila["resumen"] = combinar_textos(sub.get("resumen", pd.Series([], dtype=object)))
    fila["fuentes_presentes"] = combinar_fuentes(sub.get("fuentes_presentes", pd.Series([], dtype=object)))

    urls_unicas = combinar_urls(sub.get("urls", pd.Series([], dtype=object)))
    for i, url in enumerate(urls_unicas, start=1):
        fila[f"url_{i}"] = url

    agrupado.append(fila)

# -----------------------------
# 5. RESULTADO FINAL
# -----------------------------
base_unificada = pd.DataFrame(agrupado).sort_values("fecha_inicio").reset_index(drop=True)

# -----------------------------
# 4.1 CALCULAR DURACIÓN DESDE INICIO/FIN **solo si duracion_dias está vacía**
# -----------------------------
# Parse seguro de fechas (en caso de venir como string)
base_unificada["fecha_inicio"] = pd.to_datetime(base_unificada["fecha_inicio"], errors="coerce")
base_unificada["fecha_fin"]    = pd.to_datetime(base_unificada["fecha_fin"], errors="coerce")

# Máscara: filas donde duracion_dias está NaN y SÍ tenemos ambas fechas
mask_calc = base_unificada["duracion_dias"].isna() & base_unificada["fecha_inicio"].notna() & base_unificada["fecha_fin"].notna()

# Diferencia inclusiva: si inicio == fin -> 1 día; si no, (fin - inicio).days + 1
duracion_calc = (base_unificada.loc[mask_calc, "fecha_fin"] - base_unificada.loc[mask_calc, "fecha_inicio"]).dt.days + 1
# Evitar valores no positivos por si hay desorden en fechas (forzar mínimo 1)
duracion_calc = duracion_calc.clip(lower=1)

# Asignar solo a las filas que cumplen la máscara
base_unificada.loc[mask_calc, "duracion_dias"] = duracion_calc.astype("Int64")

# -----------------------------
# 5. EXPORTAR RESULTADO
# -----------------------------
# Salida en formato ISO
base_unificada["fecha_inicio"] = base_unificada["fecha_inicio"].dt.strftime("%Y-%m-%d")
base_unificada["fecha_fin"]    = base_unificada["fecha_fin"].dt.strftime("%Y-%m-%d")

base_unificada.to_csv("BASE_PAROS_UNIFICADOS_2013.csv", index=False, encoding="utf-8-sig")

print("Base unificada generada: BASE_PAROS_UNIFICADOS_2013.csv")
print("Paros únicos encontrados:", len(base_unificada))
print("Número máximo de columnas URL:", base_unificada.filter(like='url_').shape[1])
print("Filas con duración calculada por fechas:", mask_calc.sum())


Base unificada generada: BASE_PAROS_UNIFICADOS_2013.csv
Paros únicos encontrados: 9
Número máximo de columnas URL: 2
Filas con duración calculada por fechas: 0


In [ ]:
# ================================================================
# UNIFICAR PAROS DOCENTES  2015
# ================================================================
import pandas as pd
import numpy as np
import re
from datetime import timedelta

# -----------------------------
# 1. CARGAR BASE
# -----------------------------
df = pd.read_csv("BASE_PAROS_DOCENTES_2023_IA.csv", encoding="utf-8-sig")
df.columns = df.columns.str.strip().str.lower()

if "fecha_evento" in df.columns:
    df["fecha_evento"] = pd.to_datetime(df["fecha_evento"], errors="coerce")

df = df.dropna(subset=["fecha_evento"]).sort_values("fecha_evento").reset_index(drop=True)

# -----------------------------
# 2. FUNCIONES AUXILIARES
# -----------------------------
def combinar_textos(series):
    """Combina textos o listas de textos sin duplicar información."""
    textos = []
    for val in series.dropna():
        if isinstance(val, str):
            partes = re.split(r"[|;]+", val)
            textos.extend([p.strip() for p in partes if p.strip()])
        elif isinstance(val, (list, set, tuple)):
            textos.extend([str(p).strip() for p in val if str(p).strip()])
    return "; ".join(sorted(set(textos)))

def combinar_numeros(series):
    """Promedio de valores numéricos."""
    vals = pd.to_numeric(series, errors="coerce")
    return round(vals.mean(skipna=True), 1) if vals.notna().any() else np.nan

def combinar_fuentes(series):
    """Une todas las fuentes sin repetir."""
    return "; ".join(sorted(set(
        re.split(r"[;|,]+", ";".join(series.dropna().astype(str)))
    )))

def combinar_urls(series):
    """Combina todas las URLs únicas, creando columnas url_1, url_2, ..."""
    urls = []
    for val in series.dropna():
        if isinstance(val, str):
            for u in re.split(r"[|,;]+", val):
                if u.strip() and u.strip().startswith("http"):
                    urls.append(u.strip())
    return sorted(set(urls))

# -----------------------------
# 3. AGRUPAR POR FECHA CON TOLERANCIA DE ±2 DÍAS
# -----------------------------
# Ordenar por fecha
df = df.sort_values("fecha_evento").reset_index(drop=True)

# Crear grupos de eventos cercanos (ventana de 2 días)
grupos = []
grupo_actual = 0
fechas = df["fecha_evento"].tolist()

for i in range(len(fechas)):
    if i == 0:
        grupos.append(grupo_actual)
        continue
    diff = (fechas[i] - fechas[i - 1]).days
    if diff <= 1:
        grupos.append(grupo_actual)
    else:
        grupo_actual += 1
        grupos.append(grupo_actual)

df["grupo_paro"] = grupos

# -----------------------------
# 4. FUSIONAR INFORMACIÓN POR GRUPO
# -----------------------------
agrupado = []
for gid, sub in df.groupby("grupo_paro"):
    fila = {}
    fila["fecha_inicio"] = sub["fecha_evento"].min()
    fila["fecha_fin"] = sub["fecha_evento"].max()
    fila["ciudad"] = combinar_textos(sub.get("ciudad", pd.Series([])))
    fila["organizaciones_sindicales"] = combinar_textos(sub.get("organizaciones_sindicales", pd.Series([])))
    fila["tipo_movilizacion"] = combinar_textos(sub.get("tipo_movilizacion", pd.Series([])))
    fila["duracion_dias"] = combinar_numeros(sub.get("duracion_dias", pd.Series([])))
    fila["razones_paro"] = combinar_textos(sub.get("razones_paro", pd.Series([])))
    fila["resumen"] = combinar_textos(sub.get("resumen", pd.Series([])))
    fila["fuentes_presentes"] = combinar_fuentes(sub.get("fuentes_presentes", pd.Series([])))

    urls_unicas = combinar_urls(sub.get("urls", pd.Series([])))
    for i, url in enumerate(urls_unicas, start=1):
        fila[f"url_{i}"] = url

    agrupado.append(fila)

base_unificada = pd.DataFrame(agrupado).sort_values("fecha_inicio").reset_index(drop=True)

# -----------------------------
# 5. EXPORTAR RESULTADO
# -----------------------------
base_unificada["fecha_inicio"] = pd.to_datetime(base_unificada["fecha_inicio"], errors="coerce").dt.strftime("%Y-%m-%d")
base_unificada["fecha_fin"] = pd.to_datetime(base_unificada["fecha_fin"], errors="coerce").dt.strftime("%Y-%m-%d")

base_unificada.to_csv("BASE_PAROS_UNIFICADOS_2015.csv", index=False, encoding="utf-8-sig")

print("Base unificada generada: BASE_PAROS_UNIFICADOS_2015.csv")
print("Paros únicos encontrados:", len(base_unificada))
print("Número máximo de columnas URL:", base_unificada.filter(like='url_').shape[1])


Base unificada generada: BASE_PAROS_UNIFICADOS_2015.csv
Paros únicos encontrados: 8
Número máximo de columnas URL: 3


In [ ]:
# ================================================================
# ESTADÍSTICAS BASE PAROS UNIFICADOS
# ================================================================

base = pd.read_csv("BASE_PAROS_UNIFICADOS_2015.csv", encoding="utf-8-sig")
base.columns = base.columns.str.strip().str.lower()

# --- Fechas ---
if "fecha_inicio" in base.columns:
    base["fecha_inicio"] = pd.to_datetime(base["fecha_inicio"], errors="coerce")
if "fecha_fin" in base.columns:
    base["fecha_fin"] = pd.to_datetime(base["fecha_fin"], errors="coerce")

# --- Duración calculada si falta ---
if "duracion_dias" not in base.columns or base["duracion_dias"].isna().all():
    base["duracion_dias"] = (base["fecha_fin"] - base["fecha_inicio"]).dt.days + 1

# Asegurar valores positivos
base["duracion_dias"] = base["duracion_dias"].clip(lower=1)

# ================================================================
# 2. LIMPIEZA Y PREPARACIÓN
# ================================================================
def limpiar_texto(x):
    if pd.isna(x):
        return ""
    x = str(x).strip().lower()
    x = re.sub(r"[\[\]{}'\"]", "", x)
    return re.sub(r"\s+", " ", x).strip()

base["tipo_movilizacion"] = base["tipo_movilizacion"].apply(limpiar_texto)
base["organizaciones_sindicales"] = base["organizaciones_sindicales"].apply(limpiar_texto)
base["fuentes_presentes"] = base["fuentes_presentes"].apply(limpiar_texto)

# Mes de referencia según fecha_inicio
base["mes"] = base["fecha_inicio"].dt.to_period("M")

# ================================================================
# 3. ESTADÍSTICAS DESCRIPTIVAS
# ================================================================
print("\nGenerando estadísticas descriptivas de paros unificados...")

# ---- 3.1 Total de paros por mes
paros_mes = base["mes"].value_counts().sort_index().reset_index()
paros_mes.columns = ["Mes", "Total_Paros"]

# ---- 3.2 Tipos de movilización
tipos = (
    base["tipo_movilizacion"]
    .str.split(r"[;|,]+")
    .explode()
    .str.strip()
    .replace("", np.nan)
    .dropna()
    .value_counts()
    .reset_index()
)
tipos.columns = ["Tipo de Movilización", "Total"]

# ---- 3.3 Duración promedio, mediana y rango
base["duracion_dias"] = pd.to_numeric(base["duracion_dias"], errors="coerce")
duracion_stats = {
    "Promedio (días)": round(base["duracion_dias"].mean(skipna=True), 2),
    "Mediana (días)": base["duracion_dias"].median(skipna=True),
    "Máximo (días)": base["duracion_dias"].max(),
    "Mínimo (días)": base["duracion_dias"].min(),
    "Desviación estándar": round(base["duracion_dias"].std(skipna=True), 2)
}

# ---- 3.4 Organizaciones sindicales
def limpiar_orgs(x):
    if pd.isna(x):
        return []
    if isinstance(x, str):
        parts = re.split(r"[,;|]+", x)
        return [p.strip().upper() for p in parts if p.strip()]
    return []

orgs = (
    base["organizaciones_sindicales"]
    .apply(limpiar_orgs)
    .explode()
    .value_counts()
    .head(15)
    .reset_index()
)
orgs.columns = ["Organización Sindical", "Frecuencia"]

# ---- 3.5 Fuentes más comunes
def limpiar_fuentes(x):
    if pd.isna(x):
        return []
    if isinstance(x, str):
        return [f.strip().upper() for f in re.split(r"[,;|]+", x) if f.strip()]
    return []

fuentes = (
    base["fuentes_presentes"]
    .apply(limpiar_fuentes)
    .explode()
    .value_counts()
    .reset_index()
)
fuentes.columns = ["Fuente", "Total_Paros"]

# ---- 3.6 Rango de fechas analizado
rango_fechas = {
    "Fecha más antigua": base["fecha_inicio"].min().strftime("%Y-%m-%d"),
    "Fecha más reciente": base["fecha_fin"].max().strftime("%Y-%m-%d"),
    "Total paros": len(base)
}

# ================================================================
# 4. EXPORTAR RESULTADOS
# ================================================================
with pd.ExcelWriter("Estadisticas_Paros_Unificados_2015.xlsx", engine="openpyxl") as writer:
    paros_mes.to_excel(writer, sheet_name="Paros por Mes", index=False)
    tipos.to_excel(writer, sheet_name="Tipos de Movilización", index=False)
    orgs.to_excel(writer, sheet_name="Organizaciones", index=False)
    fuentes.to_excel(writer, sheet_name="Fuentes", index=False)
    pd.DataFrame([duracion_stats]).to_excel(writer, sheet_name="Duración", index=False)
    pd.DataFrame([rango_fechas]).to_excel(writer, sheet_name="Resumen General", index=False)

# ================================================================
# 5. RESUMEN EN CONSOLA
# ================================================================
print("\nResumen rápido:")
print(f"Total de paros únicos: {len(base)}")
print(f"Meses con paros: {base['mes'].nunique()}")
print(f"Promedio de duración: {duracion_stats['Promedio (días)']} días")
print(f"Rango de fechas: {rango_fechas['Fecha más antigua']} → {rango_fechas['Fecha más reciente']}")

print("\nArchivo generado: Estadisticas_Paros_Unificados_2015.xlsx")



Generando estadísticas descriptivas de paros unificados...

Resumen rápido:
Total de paros únicos: 8
Meses con paros: 7
Promedio de duración: 12.33 días
Rango de fechas: 2015-01-01 → 2015-07-07

Archivo generado: Estadisticas_Paros_Unificados_2015.xlsx
